In [1]:
from search_http_tool2 import SearchByHttpTool
from kgsearch_llm_obs2 import *
from get_1b_output import ApiResult
import utils


In [ ]:
 # 单结果
def search_results(search_http_tool:SearchByHttpTool, api_record : ApiRecord):
    http_tool = search_http_tool 
    retry_num = 5   # 重试次数

    multi_llm_obs = []   #返回结果

    query = api_record.user_query
    if api_record.is_valid == 0:   # 不请求搜索
        print('query=%s, api is invalid' % (query) )
        return multi_llm_obs

    for api_res in api_record.api_res_list: #多api 
        for i in range(retry_num):
            try:
                if api_record.api_name.lower() in ['qasearch', 'autosearch']:
                    tags = api_res.tag.split('&')
                    category = str(api_res.category).replace('&',',').split(',',-1)[0] #可能有多个值,取第一个值
                    _, origin_docs = http_tool.qa_search(origin_query=query, norm_query=api_res.normal_query, 
                                                        tags=tags, slots=api_res.slots, category=category,
                                                         time_slots=api_res.time_slots)
                elif api_record.api_name.lower() in ['mediasearch']:
                    _, origin_docs = http_tool.media_search(query, api_res.normal_query, api_res.slots,
                                                            time_slots=api_res.time_slots)
                multi_llm_obs.append(origin_docs)
            except Exception as e:
                print(f"Error in search: {e}")
                multi_llm_obs.append('')  # 如果出错，添加空字符串
                break
    return multi_llm_obs


def get_obs(search_http_tool, query, input_api):
    api_record = get_api_4query(query, input_api)
    multi_llm_obs = search_results(search_http_tool, api_record)
    return multi_llm_obs

In [4]:
api_record = get_api_4query(query, input_api)
api_record

In [12]:
http_tool = search_http_tool 
retry_num = 5   # 重试次数

multi_llm_obs = []   #返回结果

query = api_record.user_query
if api_record.is_valid == 0:   # 不请求搜索
    print('query=%s, api is invalid' % (query) )

for api_res in api_record.api_res_list: #多api 
    for i in range(retry_num):
        try:
            if api_record.api_name.lower() in ['qasearch', 'autosearch']:
                tags = api_res.tag.split('&')
                category = str(api_res.category).replace('&',',').split(',',-1)[0] #可能有多个值,取第一个值
                _, origin_docs = http_tool.qa_search(origin_query=query, norm_query=api_res.normal_query, 
                                                    tags=tags, slots=api_res.slots, category=category,
                                                     time_slots=api_res.time_slots)
            elif api_record.api_name.lower() in ['mediasearch']:
                _, origin_docs = http_tool.media_search(query, api_res.normal_query, api_res.slots,
                                                        time_slots=api_res.time_slots)
            multi_llm_obs.append(origin_docs)
        except Exception as e:
            print(f"Error in search: {e}")
            multi_llm_obs.append('')  # 如果出错，添加空字符串
            break

In [2]:
input_api = [{"STARTTIME":"{\"norm\":{\"now\":\"true\"},\"raw\":\"目前\"}","APINAME":"QASearch","QUERY":"目前最先进的AI芯片是哪家公司的","CATEGORY":"公司","TAG":"目前&最先进&AI芯片&公司&时效性","timeslots":"{\"type\":\"ABSOLUTE_TIME\",\"range\":\"POINT\",\"format\":\"yyyy-MM-dd HH:mm:ss\",\"start\":{\"raw\":\"目前\",\"time\":\"2024-05-24 21:01:56\",\"norm\":{\"now\":\"true\"}},\"end\":null,\"duration\":null,\"supplemental\":{\"raw\":\"\",\"time\":\"2024-05-24 21:01:56\",\"norm\":null},\"interval\":\"\"}"}]

In [3]:
input_api

[{'STARTTIME': '{"norm":{"now":"true"},"raw":"目前"}',
  'APINAME': 'QASearch',
  'QUERY': '目前最先进的AI芯片是哪家公司的',
  'CATEGORY': '公司',
  'TAG': '目前&最先进&AI芯片&公司&时效性',
  'timeslots': '{"type":"ABSOLUTE_TIME","range":"POINT","format":"yyyy-MM-dd HH:mm:ss","start":{"raw":"目前","time":"2024-05-24 21:01:56","norm":{"now":"true"}},"end":null,"duration":null,"supplemental":{"raw":"","time":"2024-05-24 21:01:56","norm":null},"interval":""}'}]